In [ ]:
# reshape only can be done on numpy array can we use tf's built-in data plz
# want to make this simplest it can be at first. how simple can 2x2 be??

In [1]:
# assignment 6
# MSDS 422
# Professor Anil Chaturvedi

# https://github.com/ageron/handson-ml/blob/master/10_introduction_to_artificial_neural_networks.ipynb
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
# https://github.com/ageron/handson-ml/blob/master/12_distributed_tensorflow.ipynb

# ??
# https://www.datacamp.com/community/tutorials/cnn-tensorflow-python

<div style="text-align: right"><b>pkg imports</b></div>

In [2]:
# basix
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import time

# prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#modeling
from sklearn.linear_model import Perceptron
import tensorflow as tf

# other

%matplotlib inline   
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<div style="text-align: right"><b>configs, table setup</b></div>

In [3]:
pd.options.display.float_format = "{:,.4f}".format
RANDOM_SEED = 8675309
responseVar = "label"

In [4]:
# setup for consolidated table, later

table = {"model 1":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 2":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 3":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 4":{"nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
        }

pd.DataFrame(table).transpose()

,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,None,None,None,None,None
model 2,None,None,None,None,None
model 3,None,None,None,None,None
model 4,None,None,None,None,None


<div style="text-align: right"><b>defining things for model</b></div>

In [5]:
# activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# lambda x: 1 / (1 + np.exp(-x))

In [ ]:
training_iters = 10
learning_rate = 0.001
batch_size = 128

# MNIST data input (img shape: 28*28)
n_input = 28

# MNIST total classes (0-9 digits)
n_classes = 10

<div style="text-align: right"><b>data setup</b></div>

In [6]:
mnist_train = pd.read_csv('train.csv')
mnist_test = pd.read_csv('test.csv')
mnist = pd.concat([mnist_test, mnist_train.drop(columns = "label")]) # combined dataset

In [7]:
X = mnist_train.loc[:, mnist_train.columns != responseVar]
y = mnist_train[responseVar]

In [8]:
# splitting og data once for simplicity
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=RANDOM_SEED)

folds = KFold(n_splits = 3, shuffle = True, random_state = RANDOM_SEED)

<div style="text-align: right"><b>model 1 : a single Perceptron. 1x1</b></div>

In [9]:
print("approximate start time : ", time.asctime())
start1 = time.perf_counter() # CPU time or real time in seconds since process start or since prev call to perf_counter(). v precise.

approximate start time :  Mon Oct 19 22:42:41 2020


In [10]:
perceptron = Perceptron(max_iter=100, tol=-np.infty, random_state=RANDOM_SEED)
perceptron.fit(X_train, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=100, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=8675309, shuffle=True, tol=-inf,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
stop1 = time.perf_counter()
tm1 = round(stop1 - start1)
print("approximate completion time, initial rfc : ", time.asctime())
print("time of execution in seconds : ", tm1)
print("time of execution in minutes : ", round(tm1/60, 1))

approximate completion time, initial rfc :  Mon Oct 19 22:43:25 2020
time of execution in seconds :  44
time of execution in minutes :  0.7


In [12]:
table["model 1"]["nbr_layers"] = "1"
table["model 1"]["nodes_p_layer"] = "1"
table["model 1"]["processing_time"] = tm1
table["model 1"]["testset_acc"] = perceptron.score(X_test, y_test)
table["model 1"]["trainset_acc"] = perceptron.score(X_train, y_train)

In [13]:
#y_preds = perceptron.predict(X_test, y_test)

<div style="text-align: right"><b>model 2 : 2x2. 2 layers w 2 neurons each</b></div>

In [14]:
print("approximate start time : ", time.asctime())
start1 = time.perf_counter()

approximate start time :  Mon Oct 19 22:43:25 2020


In [15]:
train_X = X.reshape(-1, 28, 28, 1)
test_X = y.reshape(-1, 28, 28, 1)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
twox2 = 

In [ ]:
stop2 = time.perf_counter()
tm2 = round(stop2 - start2)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm2)
print("time of execution in minutes : ", round(tm2/60, 1))

In [ ]:
table["model 2"]["nbr_layers"] = "2"
table["model 2"]["nodes_p_layer"] = "2"
table["model 2"]["processing_time"] = tm2
table["model 2"]["testset_acc"] = twox2.score(X_test, y_test)
table["model 2"]["trainset_acc"] = twox2.score(X_train, y_train)

In [ ]:
print("processing time in seconds : ")
pd.DataFrame(table).transpose()